# Programación declarativa @ URJC
# Programación funcional
## Examen simulado
## Curso 19-20

# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

### Algunas funciones sobre listas

In [ ]:
object Signatures{
    abstract class List[A]{
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def reverse: List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
    }
}

### Definiciones sobre árboles binarios

In [ ]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

In [ ]:
object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
    
    def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
        tree match {
            case Empty() => 
                empty
            case Node(left, a, right) =>
                node(foldTree(left)(empty)(node),
                    a,
                    foldTree(right)(empty)(node))
        }
}

import Tree._

### Modelo de datos de películas

In [ ]:
case class MovieDatabase(
    films: Map[Film.Id, Film],
    users: Map[User.Id, User],
    ratings: Map[(Film.Id, User.Id), Rating])
    
case class Film(
    id: Film.Id, 
    title: String, 
    director: String,
    genre: String,
    year: Int,
    country: String)

object Film{
    type Id = Int
}

case class User(
    id: User.Id,
    name: String,
    registered: Int)
        
object User{
    type Id = Int
}
        
case class Rating(
    film: Film.Id,
    user: User.Id,
    score: Int)

In [ ]:
val moviedb: MovieDatabase = MovieDatabase(
    films = Map(
        1 -> Film(1, "Blade Runner", "Ridley Scott", "Sci-Fi", 1982, "United States"),
        2 -> Film(2, "Amanece, que no es poco", "José Luis Cuerda", "Comedy", 1989, "Spain"),
        3 -> Film(3, "El milagro de P. Tinto", "Javier Fesser", "Comedy", 1998, "Spain"),
        4 -> Film(4, "Mars Attacks!", "Tim Burton", "Sci-Fi", 1996, "United States"),
        5 -> Film(5, "2001: A Space Odyssey", "Stanley Kubrick", "Sci-Fi", 1968, "United Kingdom"),
        6 -> Film(6, "El crack Cero", "José Luis Garci", "Film noir", 2019, "Spain"),
        7 -> Film(7, "El crack", "José Luis Garci", "Film noir", 1981, "Spain"),
        8 -> Film(8, "The Maltese Falcon", "John Huston", "Film noir", 1941, "United States"),
        9 -> Film(9, "Chinatown", "Roman Polanski", "Film noir", 1974, "United States"),
        10 -> Film(10, "Batman v. Superman: Dawn of Justice", "Zack Snyder", "Sci-Fi", 2016, "United States"),
        11 -> Film(11, "Dumb and Dumber", "Peter Farrelly", "Comedy", 1994, "United States")
    ),
    users = Map(
        1 -> User(1, "Juan", 1500),
        2 -> User(2, "Alf", 1555),
        3 -> User(3, "Lola", 1644),
        4 -> User(4, "Lola", 1655),
        5 -> User(5, "Dinu", 1622)),
    ratings = Map(
        (1,1) -> Rating(1,1,5),
        (1,2) -> Rating(1,2,1),
        (1,3) -> Rating(1,3,4),
        (1,4) -> Rating(1,4,3),
        (2,1) -> Rating(2,1,1),
        (2,4) -> Rating(2,4,1),
        (4,1) -> Rating(4,1,3),
        (5,4) -> Rating(5,4,2),
        (6,1) -> Rating(6,1,2),
        (7,1) -> Rating(7,1,3),
        (7,2) -> Rating(7,2,3),
        (7,3) -> Rating(7,3,3),
        (8,2) -> Rating(8,2,2),
        (9,1) -> Rating(9,1,1),
        (10,1) -> Rating(10,1,0),
        (10,3) -> Rating(10,3,0),
        (11,1) -> Rating(11,1,0),
        (11,2) -> Rating(11,2,1),
        (11,4) -> Rating(11,4,2)))

In [ ]:
object BasicQueries{
    
    // Entities
    
    def films(mdb: MovieDatabase): List[Film] =
        mdb.films.values.toList
    
    def filmIds(mdb: MovieDatabase): List[Film.Id] =
        mdb.films.keys.toList

    def getFilm(id: Film.Id)(mdb: MovieDatabase): List[Film] = 
        mdb.films.get(id).toList
    
    def userIds(mdb: MovieDatabase): List[User.Id] = 
        mdb.users.keys.toList
    
    def getUser(id: User.Id)(mdb: MovieDatabase): List[User] = 
        mdb.users.get(id).toList
    
    // 1-N relationships
    
    def films(dir: String)(mdb: MovieDatabase): List[Film.Id] = 
        mdb.films.filter(_._2.director == dir).map(_._1).toList
    
    // N-M relationships
    
    def ratings(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.values.toList
    
    def userRatings(user: User.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._2 == user).values.toList
    
    def filmRatings(film: Film.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._1 == film).values.toList
}

import BasicQueries._

# Ejercicio 1
__(1 punto)__

Implementa las siguientes funciones.

#### a) (0,25 puntos)

In [ ]:
def axiom1[P, Q, R](f: P => Q => R)(g: P => Q): P => R = 
    (p: P) => f(p)(g(p) : Q) : R

#### b) (0,25 puntos)

In [ ]:
def axiom2[P, Q, R]: 
        Either[P, Either[Q, R]] => 
        Either[Either[P, Q], R] = 
    (e: Either[P, Either[Q, R]]) => 
        e match {
            case Left(p : P) => 
                Left(Left(p) : Either[P, Q])
            case Right(qOrR: Either[Q, R]) => 
                qOrR match {
                    case Left(q: Q) => 
                        Left(Right(q))
                    case Right(r: R) =>
                        Right(r)
                }
        }

In [ ]:
def axiom2[P, Q, R]: 
        Either[P, Either[Q, R]] => 
        Either[Either[P, Q], R] = 
    {
        case Left(p : P) => 
            Left(Left(p) : Either[P, Q])
        case Right(Left(q: Q)) => 
            Left(Right(q))
        case Right(Right(r: R)) => 
            Right(r)
    }

#### c) (0,25 puntos)

In [ ]:
def axiom3[P, Q, R, S](
        f: P => Q, 
        g: R => S)(
        e: Either[P, R]): Either[Q, S] = 
    e match {
        case Left(p) => Left(f(p)) : Either[Q, S]
        case Right(r) => Right(g(r)) : Either[Q, S]
    }
    

#### d) (0,25 puntos)

In [ ]:
def axiom4[P, Q, R](f: P => Q, g: P => R): P => (Q, R) = 
    (p: P) => (f(p) : Q, g(p) : R)

# Ejercicio 2 
__(3 puntos)__

La función _penultimate_ devuelve el penúltimo elemento de una lista, en caso de que este exista. 

In [ ]:
class TestPenultimate(
    penultimate: List[Int] => Option[Int]
) extends FlatSpec with Matchers{
    
    "penultimate" should "work" in {
        penultimate(List()) shouldBe None
        penultimate(List(1)) shouldBe None
        penultimate(List(2,3)) shouldBe Some(2)
        penultimate(List(1,2,4,3,5)) shouldBe Some(3)
    }
}

#### a) (1,5 puntos)
Implementa la función de manera recursiva (no es necesario que sea recursiva por la cola).

In [ ]:
def penultimate[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head1 :: tail1 => 
            tail1 match {
                case Nil => None : Option[A]
                case head2 :: tail2 => 
                    tail2 match {
                        case Nil => Some(head1)
                        case _ =>
                            penultimate(tail1)
                    }
            }
    }

In [ ]:
def penultimate[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case _ :: Nil => 
            None : Option[A]    
        case head1 :: _ :: Nil => 
            Some(head1)
        case _ :: tail1 => 
            penultimate(tail1)
    }

#### b) (1,5 puntos)
Implementa la función utilizando `foldRight`.

In [ ]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldRight((None : Option[A], true))(
        (head: A, tailSol: (Option[A], Boolean)) => 
            tailSol match {
                case (None, true) => (None, false)
                case (None, false) => (Some(head), false)
                case (Some(p), _) => (Some(p), false)
            }
    )._1

In [ ]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldRight((None : Option[A], true)){
        case (_, (None, true)) => (None, false)
        case (head, (None, false)) => (Some(head), false)
        case (_, (Some(p), _)) => (Some(p), false)
    }._1

In [ ]:
run(new TestPenultimate(penultimate))

# Ejercicio 3
__(3 puntos)__

La función `odds` devuelve los elementos de una lista que se encuentran situados en una posición impar (donde la posición del primer elemento es uno, del segundo dos, etc.).

In [ ]:
class TestOdds(
    odds: List[Int] => List[Int]
) extends FlatSpec with Matchers{
    
    "odds" should "work" in {
        odds(List()) shouldBe List()
        odds(List(1)) shouldBe List(1)
        odds(List(2,3)) shouldBe List(2)
        odds(List(1,2,4,3,5)) shouldBe List(1,4,5)
        odds(List(1,2,4,3,5,6)) shouldBe List(1,4,5)
    }
}

#### a) (1,5 puntos)
Implementa la función de manera recursiva, utilizando recursión por la cola. 

In [ ]:
def oddsTR[A](list: List[A]): List[A] = {
    def oddsAux(out: List[A], aux: List[A]): List[A] = 
        aux match {
            case Nil => out 
            case head :: Nil => head :: out
            case head :: _ :: tail => 
                oddsAux(head :: out, tail)
        }
    oddsAux(List(), list).reverse
}

In [ ]:
def oddsTR[A](list: List[A]): List[A] = {
    def oddsAux(out: List[A], insert: Boolean, 
                aux: List[A]): List[A] = 
        aux match {
            case Nil => out 
            case head :: tail => 
                if (insert) 
                    oddsAux(head :: out, false, tail)
                else
                    oddsAux(out, true, tail)
        }
    oddsAux(List(), true, list).reverse
}

In [ ]:
run(new TestOdds(oddsTR))

#### b) (1,5 puntos)
Implementa la función utilizando la función `foldLeft`. 

In [ ]:
def oddsFL[A](list: List[A]): List[A] =
    list.foldLeft((List() : List[A], true)){
        case ((out, true), e: A) => 
            (e :: out, false)
        case ((out, _), _) => 
            (out, true)
    }._1.reverse

In [ ]:
run(new TestOdds(oddsFL))

# Ejercicio 4
(1,5 puntos)

Un árbol se encuentra _balanceado_ si (1) es el árbol vacío o (2) si sus hijos izquierdo y derecho se encuentran balanceados, y la diferencia entre sus profundidades no excede la unidad. La profundidad de un árbol vacío es cero, y la de un árbol no vacío el máximo de las profundidades de sus hijos más uno.

In [ ]:
class TestBalanced(
    balanced: Tree[Int] => Boolean
) extends FlatSpec with Matchers {
    
    "balanced" should "work" in {
        balanced(void) shouldBe true
        balanced(leaf(1)) shouldBe true
        balanced(left(leaf(1), 2)) shouldBe true
        balanced(node(leaf(1), 2, leaf(3))) shouldBe true
        balanced(node(left(leaf(1),4), 2, leaf(3))) shouldBe true
        balanced(right(3, left(leaf(1),4))) shouldBe false
        balanced(node(right(3, left(leaf(1),4)),5,leaf(3))) shouldBe false
    }
}

Se pide implementar una función que determine si un árbol está balanceado o no, utilizando la función auxiliar `foldTree`.

In [ ]:
def balanced[A](tree: Tree[A]): Boolean = 
    foldTree(tree)((true : Boolean, 0: Int))(
        (solL: (Boolean, Int), 
         root, 
         solR: (Boolean, Int)) => 
            (solL._1 && solR._1 && 
              ((solL._2 - solR._2).abs <= 1) : Boolean,
             1 + solL._2 max solR._2)
    )._1

In [ ]:
def balanced[A](tree: Tree[A]): Boolean = 
    foldTree(tree)((true : Boolean, 0: Int)){
        case ((balancedL, depthL), root, (balancedR, depthR)) => 
            (balancedL && balancedR && (depthL - depthR).abs <= 1,
             1 + depthL max depthR)
    }._1

In [ ]:
run(new TestBalanced(balanced))

# Ejercicio 5
(1,5 puntos)

Dado el modelo de datos de películas, implementar una query que devuelva los títulos de las películas cuyas puntuaciones (`score`) sean todas superiores o iguales a tres.

In [ ]:
class TestTopMovies(
    topMovies: MovieDatabase => List[String]
) extends FlatSpec with Matchers {
    
    "topMovies" should "work" in {
        topMovies(moviedb) shouldBe List(
            "El crack",
            "El milagro de P. Tinto",
            "Mars Attacks!")
    }
}

In [ ]:
def topMovies(mdb: MovieDatabase): List[String] = 
    films(mdb).map(film => 
        (film.title, filmRatings(film.id)(mdb))
    ).filter(tuple => 
       tuple._2.forall(rating => rating.score >= 3) 
    ).map(tuple => tuple._1)

In [ ]:
def topMovies(mdb: MovieDatabase): List[String] = 
    films(mdb).filter(film => 
        filmRatings(film.id)(mdb).forall(_.score >= 3)
    ).map(_.title)

In [ ]:
run(new TestTopMovies(topMovies))